In [2]:
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()
import numpy as np

# No variable sharing
This would cause duplicated variables.

In [36]:
image1 = np.random.random([100, 10, 10, 3]).astype(np.float32)
def inference(image):
    # conv1
    conv1_weights = tf.Variable(tf.truncated_normal([2, 2, 3, 5]),
                               name='conv1_weights')
    conv1_biases = tf.Variable(tf.zeros([5]),
                              name='conv1_biases')
    conv1 = tf.nn.conv2d(image, conv1_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + conv1_biases)
    print(relu1.get_shape())
    
    # conv2
    conv2_weights = tf.Variable(tf.truncated_normal([2, 2, 5, 6]),
                               name='conv2_weights')
    conv2_biases = tf.Variable(tf.zeros([6]),
                              name='conv2_biases')
    conv2 = tf.nn.conv2d(relu1, conv2_weights, strides=[1, 1, 1, 1], padding='SAME')
    relu2 = tf.nn.relu(conv2 + conv2_biases)
    print(relu2.get_shape())
    return relu2

In [51]:
tf.reset_default_graph()
inference(image1)
inference(image1 + 2)
[x._variable for x in tf.global_variables()]

(100, 10, 10, 5)
(100, 10, 10, 6)
(100, 10, 10, 5)
(100, 10, 10, 6)


[<tf.Tensor 'conv1_weights:0' shape=(2, 2, 3, 5) dtype=float32_ref>,
 <tf.Tensor 'conv1_biases:0' shape=(5,) dtype=float32_ref>,
 <tf.Tensor 'conv2_weights:0' shape=(2, 2, 5, 6) dtype=float32_ref>,
 <tf.Tensor 'conv2_biases:0' shape=(6,) dtype=float32_ref>,
 <tf.Tensor 'conv1_weights_1:0' shape=(2, 2, 3, 5) dtype=float32_ref>,
 <tf.Tensor 'conv1_biases_1:0' shape=(5,) dtype=float32_ref>,
 <tf.Tensor 'conv2_weights_1:0' shape=(2, 2, 5, 6) dtype=float32_ref>,
 <tf.Tensor 'conv2_biases_1:0' shape=(6,) dtype=float32_ref>]

# Variable sharing through dictionary

In [43]:
variable_dict = {
    'conv1_weights': tf.Variable(tf.truncated_normal([2, 2, 3, 5]),
                               name='conv1_weights'),
    'conv1_biases': tf.Variable(tf.zeros([5]),
                              name='conv1_biases'),
    'conv2_weights': tf.Variable(tf.truncated_normal([2, 2, 5, 6]),
                               name='conv2_weights'),
    'conv2_biases': tf.Variable(tf.zeros([6]),
                              name='conv12biases')
    }
def inference_with_dict(image):
    conv1 = tf.nn.conv2d(image, variable_dict['conv1_weights'], strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + variable_dict['conv1_biases'])
    print(relu1.get_shape())

    conv2 = tf.nn.conv2d(relu1, variable_dict['conv2_weights'], strides=[1, 1, 1, 1], padding='SAME')
    relu2 = tf.nn.relu(conv2 + variable_dict['conv2_biases'])
    print(relu2.get_shape())
    return relu2

In [58]:
tf.reset_default_graph()
inference_with_dict(image1)
inference_with_dict(image1 + 2)
[x._variable for x in tf.global_variables()]
variable_dict

(100, 10, 10, 5)
(100, 10, 10, 6)
(100, 10, 10, 5)
(100, 10, 10, 6)


{'conv1_biases': <tensorflow.python.ops.variables.Variable at 0x10cbd9150>,
 'conv1_weights': <tensorflow.python.ops.variables.Variable at 0x10cbd90d0>,
 'conv2_biases': <tensorflow.python.ops.variables.Variable at 0x10cc02dd0>,
 'conv2_weights': <tensorflow.python.ops.variables.Variable at 0x10cbf6b10>}

# This breaks encapsulation. Try `get_variable()`!

In [92]:
def get_relu(image, kernel_shape, bias_shape):
    weights = tf.get_variable('weights', kernel_shape,
                          initializer=tf.random_normal_initializer())
    biases = tf.get_variable('biases', bias_shape,
                          initializer=tf.constant_initializer(0.0))

    conv = tf.nn.conv2d(image, weights, strides=[1, 1, 1, 1], padding='SAME')
    relu = tf.nn.relu(conv + biases)
    return relu

    
def inference_with_get_var(image):
    with tf.variable_scope('conv1'):
        relu1 = get_relu(image, [2, 2, 3, 5], [5])
        print(relu1.get_shape())
    print('conv1 ===> conv2')
    with tf.variable_scope('conv2'):
        relu2 = get_relu(relu1, [2, 2, 5, 6], [6])
        print(relu2.get_shape())
    return relu2

In [93]:
tf.reset_default_graph()
with tf.variable_scope('image1'):
    inference_with_get_var(image1)
#     tf.get_variable_scope().reuse_variables()
with tf.variable_scope('image2'):
    inference_with_get_var(image1 + 2)
[x._variable for x in tf.global_variables()]

(100, 10, 10, 5)
conv1 ===> conv2
(100, 10, 10, 6)
(100, 10, 10, 5)
conv1 ===> conv2
(100, 10, 10, 6)


[<tf.Tensor 'image1/conv1/weights:0' shape=(2, 2, 3, 5) dtype=float32_ref>,
 <tf.Tensor 'image1/conv1/biases:0' shape=(5,) dtype=float32_ref>,
 <tf.Tensor 'image1/conv2/weights:0' shape=(2, 2, 5, 6) dtype=float32_ref>,
 <tf.Tensor 'image1/conv2/biases:0' shape=(6,) dtype=float32_ref>,
 <tf.Tensor 'image2/conv1/weights:0' shape=(2, 2, 3, 5) dtype=float32_ref>,
 <tf.Tensor 'image2/conv1/biases:0' shape=(5,) dtype=float32_ref>,
 <tf.Tensor 'image2/conv2/weights:0' shape=(2, 2, 5, 6) dtype=float32_ref>,
 <tf.Tensor 'image2/conv2/biases:0' shape=(6,) dtype=float32_ref>]

In [94]:
tf.reset_default_graph()
with tf.variable_scope('image1') as scope:
    inference_with_get_var(image1)
    scope.reuse_variables()
    assert scope == tf.get_variable_scope()
    inference_with_get_var(image1 + 2)
[x._variable for x in tf.global_variables()]

(100, 10, 10, 5)
conv1 ===> conv2
(100, 10, 10, 6)
(100, 10, 10, 5)
conv1 ===> conv2
(100, 10, 10, 6)


[<tf.Tensor 'image1/conv1/weights:0' shape=(2, 2, 3, 5) dtype=float32_ref>,
 <tf.Tensor 'image1/conv1/biases:0' shape=(5,) dtype=float32_ref>,
 <tf.Tensor 'image1/conv2/weights:0' shape=(2, 2, 5, 6) dtype=float32_ref>,
 <tf.Tensor 'image1/conv2/biases:0' shape=(6,) dtype=float32_ref>]